<a href="https://colab.research.google.com/github/advik-7/NLP_projects/blob/main/QnA_chatbot_practise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pickle

In [2]:
import numpy as np

In [8]:
with open("/content/train_qa.txt",'rb') as f:
  train_data= pickle.load(f)

In [9]:
with open("/content/test_qa.txt",'rb') as f:
  test_data= pickle.load(f)

In [10]:
type(test_data)

list

In [11]:
len(train_data)

10000

In [12]:
len(test_data)

1000

In [13]:
train_data[0]

(['Mary',
  'moved',
  'to',
  'the',
  'bathroom',
  '.',
  'Sandra',
  'journeyed',
  'to',
  'the',
  'bedroom',
  '.'],
 ['Is', 'Sandra', 'in', 'the', 'hallway', '?'],
 'no')

In [14]:
" ".join(train_data[0][0])

'Mary moved to the bathroom . Sandra journeyed to the bedroom .'

In [15]:
" ".join(train_data[0][1])

'Is Sandra in the hallway ?'

In [16]:
" ".join(train_data[0][2])

'n o'

In [17]:
data = train_data+test_data

In [18]:
len(data)

11000

In [19]:
vocab=set()
for story,question,answer in data:
  vocab=vocab.union(set(story))
  vocab=vocab.union(set(question))
  vocab=vocab.union(set(answer))

In [20]:
vocab.add("yes")
vocab.add("no")

In [21]:
vocab_len= len(vocab)+1

In [22]:
vocab.discard("s")

In [23]:
vocab.discard("o")
vocab.discard("n")
vocab.discard("y")

In [24]:
vocab

{'.',
 '?',
 'Daniel',
 'Is',
 'John',
 'Mary',
 'Sandra',
 'apple',
 'back',
 'bathroom',
 'bedroom',
 'discarded',
 'down',
 'dropped',
 'e',
 'football',
 'garden',
 'got',
 'grabbed',
 'hallway',
 'in',
 'journeyed',
 'kitchen',
 'left',
 'milk',
 'moved',
 'no',
 'office',
 'picked',
 'put',
 'the',
 'there',
 'to',
 'took',
 'travelled',
 'up',
 'went',
 'yes'}

In [25]:
vocab_len

43

In [26]:
all_story_len=[len(data[0]) for data in data]

In [27]:
max_story_len = max(all_story_len)

In [28]:
max_question_len=max([len(data[1]) for data in data])

In [29]:
max_question_len

6

In [30]:
!pip install tensorflow



In [31]:
 from keras.preprocessing.sequence import pad_sequences

In [32]:
from tensorflow.keras.preprocessing.text import Tokenizer


In [33]:
tokenizer=Tokenizer(filters=[])
tokenizer.fit_on_texts(vocab)

In [34]:
tokenizer.word_index

{'up': 1,
 'to': 2,
 'bathroom': 3,
 'garden': 4,
 'grabbed': 5,
 'journeyed': 6,
 'yes': 7,
 'put': 8,
 'down': 9,
 'bedroom': 10,
 '.': 11,
 'football': 12,
 'e': 13,
 'dropped': 14,
 'left': 15,
 'kitchen': 16,
 'discarded': 17,
 'mary': 18,
 'apple': 19,
 'is': 20,
 'john': 21,
 'office': 22,
 'milk': 23,
 'travelled': 24,
 'back': 25,
 'got': 26,
 'took': 27,
 'hallway': 28,
 '?': 29,
 'went': 30,
 'there': 31,
 'no': 32,
 'moved': 33,
 'picked': 34,
 'the': 35,
 'in': 36,
 'daniel': 37,
 'sandra': 38}

In [35]:
train_story_text=[]
train_question_text=[]
train_answers=[]
for story,question,answers in train_data:
  train_story_text.append(story)
  train_question_text.append(question)
  train_answers.append(answers)


In [36]:
train_story_text

[['Mary',
  'moved',
  'to',
  'the',
  'bathroom',
  '.',
  'Sandra',
  'journeyed',
  'to',
  'the',
  'bedroom',
  '.'],
 ['Mary',
  'moved',
  'to',
  'the',
  'bathroom',
  '.',
  'Sandra',
  'journeyed',
  'to',
  'the',
  'bedroom',
  '.',
  'Mary',
  'went',
  'back',
  'to',
  'the',
  'bedroom',
  '.',
  'Daniel',
  'went',
  'back',
  'to',
  'the',
  'hallway',
  '.'],
 ['Mary',
  'moved',
  'to',
  'the',
  'bathroom',
  '.',
  'Sandra',
  'journeyed',
  'to',
  'the',
  'bedroom',
  '.',
  'Mary',
  'went',
  'back',
  'to',
  'the',
  'bedroom',
  '.',
  'Daniel',
  'went',
  'back',
  'to',
  'the',
  'hallway',
  '.',
  'Sandra',
  'went',
  'to',
  'the',
  'kitchen',
  '.',
  'Daniel',
  'went',
  'back',
  'to',
  'the',
  'bathroom',
  '.'],
 ['Mary',
  'moved',
  'to',
  'the',
  'bathroom',
  '.',
  'Sandra',
  'journeyed',
  'to',
  'the',
  'bedroom',
  '.',
  'Mary',
  'went',
  'back',
  'to',
  'the',
  'bedroom',
  '.',
  'Daniel',
  'went',
  'back',
  'to

In [37]:
def vectorize_story(data,word_index=tokenizer.word_index,):
  X=[]
  Xb=[]
  Y=[]

  for story,query,answer in data:
    x=[word_index[word.lower()]for word in story]

    xb=[word_index[word.lower()]for word in query]
    y=np.zeros(len(word_index)+1)

        # Set the position of the answer to 1
    answer_index = word_index.get(answer.lower(), -1)  # Use -1 if the answer is not in the word_index
    if answer_index != -1:
      y[answer_index + 1] = 1  # +1 to account for the extra class
    X.append(x)
    Y.append(y)
    Xb.append(xb)
    return  (pad_sequences(X,maxlen=max_story_len),pad_sequences(Xb,maxlen=max_question_len),np.array(Y))

In [38]:
inputs_train,queries_train,answers_train = vectorize_story(train_data)

In [39]:
inputs_test,queries_test,answers_test = vectorize_story(test_data)

In [66]:
inputs_test

array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        18, 26, 35, 23, 31, 11, 21, 33,  2, 35, 10, 11]], dtype=int32)

In [40]:
inputs_train

array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        18, 33,  2, 35,  3, 11, 38,  6,  2, 35, 10, 11]], dtype=int32)

In [41]:
answers_train

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 1., 0., 0., 0., 0., 0.]])

In [42]:
from keras.models import Sequential,Model

In [43]:
from tensorflow.keras.layers import Embedding


In [44]:
from keras.layers import Input,Activation,Dense,Permute,Dropout,add,dot,concatenate,LSTM

In [49]:
#placeholder
input_sequence=Input((max_story_len,))
question=Input((max_question_len,))


In [50]:
vocab_size=len(vocab)+1

In [51]:
#encoders
input_encoder_m = Sequential()
input_encoder_m.add(Embedding(input_dim=vocab_size,output_dim=64))
input_encoder_m.add(Dropout(0.3))

In [52]:
input_encoder_c = Sequential()
input_encoder_c.add(Embedding(input_dim=vocab_size,output_dim=max_question_len))
input_encoder_c.add(Dropout(0.3))

In [55]:
question_encoder=Sequential()
question_encoder.add(Embedding(input_dim=vocab_size,output_dim=64,input_length=max_question_len))
question_encoder.add(Dropout(0.3))


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [57]:
input_encoded_m=input_encoder_m(input_sequence)
input_encoded_c=input_encoder_c(input_sequence)
question_encoded=question_encoder(question)

In [58]:
match =dot([input_encoded_m,question_encoded],axes=(2,2))
match = Activation('softmax')(match)

In [59]:
response=add([match,input_encoded_c])
response=Permute((2,1))(response)
answer=concatenate([response,question_encoded])

In [60]:
answer=LSTM(32)(answer)

In [61]:
answer=Dropout(0.5)(answer)
answer=Dense(vocab_size)(answer)
answer=Activation('softmax')(answer)

In [62]:
model=Model([input_sequence,question],answer)
model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])

In [64]:
model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2             │ (None, 156)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_3             │ (None, 6)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ sequential_2 (Sequential) │ (None, 156, 64)        │          2,496 │ input_layer_2[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ sequential_6 (Sequential) │ (None, 6, 64)          │          2,496 │ input_layer_3[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dot (Dot)                 │ (None, 156, 6)         │              0 │ sequential_2[1][0],    │
│                           │                        │                │ sequential_6[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation (Activation)   │ (None, 156, 6)         │              0 │ dot[0][0]              │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ sequential_3 (Sequential) │ (None, 156, 6)         │            234 │ input_layer_2[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add (Add)                 │ (None, 156, 6)         │              0 │ activation[0][0],      │
│                           │                        │                │ sequential_3[1][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ permute (Permute)         │ (None, 6, 156)         │              0 │ add[0][0]              │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate (Concatenate) │ (None, 6, 220)         │              0 │ permute[0][0],         │
│                           │                        │                │ sequential_6[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm (LSTM)               │ (None, 32)             │         32,384 │ concatenate[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_3 (Dropout)       │ (None, 32)             │              0 │ lstm[0][0]             │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 39)             │          1,287 │ dropout_3[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_1 (Activation) │ (None, 39)             │              0 │ dense[0][0]            │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 38,897 (151.94 KB)

 Trainable params: 38,897 (151.94 KB)

 Non-trainable params: 0 (0.00 B)

In [65]:
history=model.fit([inputs_train,queries_train],answers_train,batch_size=32,epochs=50,validation_data=([inputs_test,queries_test],answers_test))

Epoch 1/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - accuracy: 0.0000e+00 - loss: 3.4145 - val_accuracy: 0.0000e+00 - val_loss: 3.4805
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - accuracy: 0.0000e+00 - loss: 3.6780 - val_accuracy: 1.0000 - val_loss: 3.3089
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.0000e+00 - loss: 3.3539 - val_accuracy: 1.0000 - val_loss: 3.2490
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 1.0000 - loss: 2.8515 - val_accuracy: 1.0000 - val_loss: 3.0647
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 1.0000 - loss: 2.8169 - val_accuracy: 1.0000 - val_loss: 2.9225
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step - accuracy: 1.0000 - loss: 2.8437 - val_accuracy: 1.0000 - val_loss: 2.7268
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 1.0000 - loss: 2.4746 - val_accuracy: 1.0000 - val_loss: 2.5340
Epoch 8/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step - accuracy: 0.0000e+00 - loss: 3.1036 - val_accuracy: 1.0000